In [ ]:
from sctoolbox.utils.jupyter import bgcolor, _compare_version

# change the background of input cells
bgcolor("PowderBlue", select= [3, 6, 10, 20, 25, 34])

nb_name = "02_QC_filtering.ipynb"

_compare_version(nb_name)

In [ ]:
import os 
import importlib
import tempfile

#set tempdir 
os.environ["TMPDIR"] = "/mnt/workspace_stud/stud4/stud4/tmpdir"

#reload module
importlib.reload(tempfile)

#print tempdir 
print(tempfile.gettempdir())

# 02 - QC and filtering
<hr style="border:2px solid black"> </hr>

## 1 - Description

**Quality control**

We must ensure that all cellular barcode data correspond to viable cells.

To ensure this quality control (QC) is mandatory and for ATAC-seq data based on 4 key aspects:

1. The signal-to-noise, either via i) the enrichment of known regions, or ii) the determination of the ratio of fragments in peaks (FRiP).
2. The total number of unique fragments, also known as library complexity.
3. The fraction of reads derived from mitochondrial DNA vs. nuclear DNA.
4. The Fragment Length Distribution.

**DOI: [10.1186/s13059-020-1929-3](https://doi.org/10.1186/s13059-020-1929-3)**

On the single cell scale additional aspects, such as multiplets have to be taken into account.

**DOI: [10.1038/s41467-021-21583-9](https://doi.org/10.1038/s41467-021-21583-9)**

Based on QC related columns stored in the .obs we can filter for high quality cells based on all these aspects in this notebook.

**Feature Selection**

For subsequent processing steps such as dimension reduction, embedding and clustering, the filtering of features and the selection of highly variable features can be conducive.

Therefore this notebooks provides steps to exclude regions from sex chromosomes and mitochondrial chromosomes and to select highly variabele features.

____________

## 2 - Setup

In [ ]:
# sctoolbox modules
import sctoolbox
import sctoolbox.tools.qc_filter as qc
import sctoolbox.utils as utils
import sctoolbox.tools as tools
import sctoolbox.plotting as pl

import peakqc.fld_scoring as fld
import matplotlib.pyplot as plt
import episcanpy as epi
import pandas as pd
import scrublet as scr
from pathlib import Path

sctoolbox.settings.settings_from_config("config.yaml", key="02")

_____________

## 3 - Load anndata
Uses the anndata object written by the previous notebook.

**Checkpoint**  
Before the filtering is performed, some ATAC-specific QC metrics are calculated. This may take some time.  
To speed up the process of redoing the QC filtering, a checkpoint file is saved after the metrics are calculated, which is used by default if it exists.

To disable the use of the checkpoint file, set `use_checkpoint` to `False`. If no checkpoint file is found, it is automatically disabled.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
use_checkpoint = True # Use checkpoint file if True

---

In [ ]:
# Checkpoint file
checkpoint = "anndata_qc_metric_checkpoint.h5ad"
# Set checkpoint as Path to enable checks
checkpoint_file = Path(f"{sctoolbox.settings.adata_output_dir}/{checkpoint}")
if use_checkpoint and not checkpoint_file.is_file():
    print("No checkpoint found. Enabling the calculation of the QC metrics.")
    use_checkpoint = False

In [ ]:
adata_name = checkpoint if use_checkpoint else "anndata_1.h5ad"
adata = utils.adata.load_h5ad(adata_name)

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

____________

## 4 - QC and filtering
<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Set the column in adata.obs containing the biological condition to evaluate
condition_column = "meta c19_severity"

# Absolute minimum number of features for pre-selection of cells before QC 
min_genes = 1

# Choose whether to binarize the X matrix
binarize_mtx = False  # True or False; convert matrix to binary

# Optional: Plot STARsolo quality if a path is given
quant_folder = ""

# correction of ambient signal using scAR",
# Caution this process is expensive and thus will take time to run!",
# Requires the raw (unfiltered) AnnData object containing all droplets.",
path_raw_adata = ""  # The path to the raw h5ad file. Leave empty to skip.",
epochs = 150  # Number of iterations for the model."

#----------------------- Doublet removal ------------------------

# Use native scrublet or the scanpy wrapper (scanpy: optimized for RNA)
use_native_scrublet = True

# Default threshold to apply doublet removal on (None for automatic threshold)
doublet_threshold = None

# Available threads
threads = 2

----------------------

In [ ]:
if not use_checkpoint:
    # Ensure that condition column is a category
    adata.obs[condition_column] = adata.obs[condition_column].astype("category")

### 4.1 - Show STARsolo quality (optional)

If the data was mapped using STARsolo, use the parameter to set the path to the STARsolo runs and plot quality measures across runs. The path must be a folder, e.g. "path/to/starsolo_output", which contains folders per condition e.g. "cond1", "cond2", etc.

In [ ]:
if quant_folder != "":
    _ = pl.qc_filter.plot_starsolo_quality(quant_folder, save="starsolo_quality.pdf")
    _ = pl.qc_filter.plot_starsolo_UMI(quant_folder, ncol=3, save="starsolo_cell_selection.pdf")

_______

### 4.2 - Remove empty cells and features

In [ ]:
if not use_checkpoint:
    print('original shape:')
    print(adata.shape)
    print('Removing empty features and cells...')

    adata = adata[adata.X.sum(axis=1) > 0]
    adata = adata[:, adata.X.sum(axis=0) > 0]

print('new shape:')
print(adata.shape)

________

### 4.3 Add ATAC specific metrices
<hr style="border:1px solid black"> </hr>

Add ATAC specific QC-metrics to the `.obs` table.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
## 1. Source of fragments

# Either provide a bamfile or a bedfile containing fragments
fragments_file = "/mnt/workspace_stud/stud6/atac/napkon_fragment/napkon_fragments.bed"

# Name of the bam-tag which contains the barcode information (usually 'CB')
barcode_tag = '3'

## 2. Choose actions to be done

# 2.1 calculate fragment length distribution score
calculate_fld_score = True

# 2.2 calculate overlap between fragments and regions
calculate_overlap = True

# Additional settings for the overlap
region_name = ''
regions_file = '/mnt/workspace_stud/napkon_data/wp1_atac/homo_sapiens.104.promoters2000.gtf'
genes_gtf = '/mnt/workspace_stud/stud8/gencode.v47.basic.annotation.gtf'

# Number of threads available
threads = 8

---

#### 4.3.1 - Check barcode tag 
If a bamfile is provided this checks if the barcodes are available in the anndata object

In [ ]:
use_bam = fragments_file.endswith("bam")
if use_bam and not use_checkpoint:
    tools.bam.check_barcode_tag(adata, fragments_file, barcode_tag)

---

#### 4.3.2 - Score fragment length distributions (FLD Score)
This ATAC specific quality control metric scores the fragment length distribution pattern of individual cells. This step utilizes PEAKQC for the score calculation. Choose if this score should be calculated and provide a file containing the fragments of the features. As input a bamfile with the raw reads or a bedfile containing fragments are suitable.

In [ ]:
if calculate_fld_score and not use_checkpoint:
    fld.add_fld_metrics(adata=adata,
                        fragments=fragments_file,
                        barcode_col=None,
                        barcode_tag=barcode_tag,
                        chunk_size_bam=1000000,
                        regions=None,
                        peaks_thr=10,
                        wavelength=150,
                        sigma=0.4,
                        plot=False,
                        save_density=None,
                        save_overview=None,
                        sample=0)

    adata.obs

---

#### 4.3.3 - Calculate an overlap
Compares the amount of fragments within the provided regions (e.g. promoters) against the amount of fragments located in the remaining cell.

In [ ]:
#if calculate_overlap and not use_checkpoint:
    #tools.calc_overlap_fc.fc_fragments_in_regions(
        #adata=adata,
        #regions_file=regions_file,
        #bam_file=fragments_file if use_bam else None,
        #fragments_file=fragments_file if not use_bam else None,
        #cb_col=None,
        #cb_tag=barcode_tag,
        #regions_name=region_name,
        #threads=threads,
        #temp_dir=None
    )

    #adata.obs

---

#### 4.3.4 - Calculate fraction of reads in peaks (FRiP Score)
The Fraction of reads in Peaks (FRiP Score) can be used to evaluate the overall signal-to-noise ratio. It provides information about the fraction of reads located within the called peak.

In [ ]:
#if not use_checkpoint:
    #adata, total_frip = tools.frip.calc_frip_scores(adata, 
                                                    #fragments_file, 
                                                    #temp_dir='')
    #adata.obs

---

#### 4.3.5 - Calculate transcription start site enrichment (TSSe)
As for the FRiP Score the Transcription Start Site Enrichment (TSSE) Score is used to evaluate the signal-to-noise ratio. It is a score that describes the proximity of signal to transcription start sites.

In [ ]:
#if not use_checkpoint:
    #adata, tSSe_df = tools.tsse.add_tsse_score(adata,
                                               #fragments=fragments_file,
                                               #gtf=genes_gtf,
                                               #negativ_shift=2000,
                                                #positiv_shift=2000,
                                               #edge_size_total=100,
                                               #edge_size_per_base=50,
                                               #min_bias=1.0,
                                               #keep_tmp=False,
                                               #temp_dir="",
                                               #plot=True,
                                               #return_aggs=True)

#### Save checkpoint file

In [ ]:
if not use_checkpoint:
    utils.adata.save_h5ad(adata, checkpoint)

---

### 4.4 - Denoising
Remove ambient signal and technical noise from the count matrix using [scAR](https://www.biorxiv.org/content/10.1101/2022.01.14.476312v4). The tool estimates the ambient profile by averaging cell-free droplets. An autoencoder neural network later corrects the count matrix.

In [ ]:
import scanpy as sc

if path_raw_adata:
    print("Loading raw anndata...")
    adata_raw = sc.read_h5ad(path_raw_adata)
    print("Denoising data, this will take a while...")
    adata = qc.denoise_data(adata, adata_raw, feature_type='Peaks', epochs=epochs,
                            verbose=False, save='droplets_kneeplot.pdf')

_______

### 4.5 - Binarize matrix
Binarizes the count matrix to be either 0 or 1. This corresponds to the general view that chromatin is either accessible (1) or inaccesible (0). However, it is still under debate whether binarization may result in a loss of information. Therefore,  binarization is optional.

**DOI: [10.1038/s41592-023-02112-6](https://doi.org/10.1038/s41592-023-02112-6)**

In [ ]:
# save raw matrix
adata.layers["raw"] = adata.X.copy()
# binarize
if binarize_mtx:
    epi.pp.binarize(adata)

_______

### 4.6 Calculate and remove doublets
Doublets are artifacts where two (doublet) or more (multiplet) cells receive the same barcode. As multiplets behave as a joined feature set of the collected cells they may show up as a separate group in downstream analysis, thus potentially skewing results. Therefore, it is recommended to remove doublets.

**DOI: [10.1016/j.cels.2018.11.005](https://doi.org/10.1016/j.cels.2018.11.005)**

In [ ]:
#tools.qc_filter.estimate_doublets(adata, use_native=use_native_scrublet, groupby=condition_column, threads=threads, threshold=doublet_threshold)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Remove predicted doublet
remove_doublets = False

---

In [ ]:
if remove_doublets:
    # Remove the duplicates from adata
    tools.qc_filter.filter_cells(adata, "predicted_doublet", name="doublet")

In [ ]:
# remove empty features
adata = adata[adata.X.sum(axis=1) > 0]
adata = adata[:, adata.X.sum(axis=0) > 0]

______

### 4.7 - Cell filtering
<hr style="border:1px solid black"> </hr>

Identify low quality cells and remove them from the dataset. Low quality cells are investigated using several metrics, which can be choosen below.

In [ ]:
# Recalculate standard QC metrics (counts...)
adata = tools.qc_filter.calculate_qc_metrics(adata, var_type='features')

# drop total_counts as it is the same as n_features
adata.obs.drop(columns=["total_counts", "log1p_total_counts"], inplace=True)

# Remove peaks with 0 count
zero_bool = adata.var["n_cells_by_counts"] == 0
adata = adata[:,~zero_bool]

In [ ]:
# available obs columns
with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata.obs)

Any numeric column shown above can be used as filter metric. Here is a description of the commonly available metrics:

| Metric | Description | Recommendation |
|--------|-------------|----------------|
|fld_score|A score describing the fit of the fragment length distribution to the expected nucleosome driven fragment length distribution.|Bigger values are better.|
|mean_fragment_size|The mean size of the DNA fragments of each cell.|Good quality data shows an enrichment for nucleosome free (<147bp) or low number nucleosome fragments.|
|n_fragments|The number of DNA fragments per cell.|A lower bound should consider the trade of between a more lenient approach that will keep rarer cell types and low quality cells alike and a stricter threshold that filters more of both groups. The upper threshold should filter outliers as these are likely to be artifacts. Should correlate with `n_features`. |
|fold_change_promoters_fragments|The comparison of fragments within promoter regions vs. outside of promoter regions.|A higher value (less fragments outside promoters) is preferable as fragments outside of promoters are considered noise.|
|frip|Fraction of reads in peaks. The fraction of reads that are located within the called peaks (0-1).|Similar to `fold_change_promoters_fragments` higher is better as a high number of reads outside of the called peaks is considered a sign for bad quality.|
|tsse_score|Transcription start site enrichment score in other words a score that describes the proximity of signal to transcription start sites.|More is better. However, extremely high values should be filtered as they are likely artifacts. The lower bound should be `>0` to remove low quality cells.|
|n_features|The number of features (peaks) found within the respective cell. This should correlate with `n_fragments` (the detected DNA fragments per cell).|Needs similar consideration as `n_fragments`.|
|log1p_n_features|Same as n_features but on a logarithmic scale.|See above.|

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Decide whether to estimate thresholds individual per condition (False) or globally (True)
global_threshold = True

# Before filtering the impact of the individual filters are plotted by an UpSet plot.
# To restrict complexity of the plot the plotted combinations can be limited below.
limit_combinations = 2 # Either provide the combination grade as Integer or None to include all

# Set initial filter thresholds
# The thresholds can be interctively changed later on
# Note: Only metrics provided below are available for filtering 
 
default_thresholds = {
    'n_features': {'min': 33000, 'max': 40000},
    #'log1p_n_features': {'min': None, 'max': None},
    #'fld_score': {'min': 3000000, 'max': 20000000},
    'fld_score_wp3': {'min': 2675, 'max': 4900}, # Aus Analyse von WP3 
    #'frip': {'min': None, 'max': None},
    #'tsse_score': {'min': 0, 'max': None},
    'n_fragments_wp3': {'min': 3000000, 'max': 60000000},
    'mean_fragment_size_wp3': {'min': 50, 'max': 290},
    'atac fraction of reads in peaks (frip)':{'min': 0.0125, 'max': 0.15},
    'fold_change_promoters_fragments': {'min': 0.03 , 'max': 0.12}
    # add additional threshold based on the available columns shown above
    # format: '<obs clolumn>': {'min': <threshold|None>, 'max': <threshold|None>}
    # None = automatically derive initial threshold
    # float('inf') or float('-inf') = no filter
}

__________

#### 4.7.1 Adapt thresholds

In [ ]:
groupby = condition_column if global_threshold is False else None
initial_thresholds = tools.qc_filter.get_thresholds(adata,
                                                    default_thresholds, 
                                                    groupby=groupby)
obs_columns = list(initial_thresholds.keys())
tools.qc_filter.thresholds_as_table(initial_thresholds)

The plot below estimates the impact each metric (and combination of metrics) would have on the data. Metrics that filter the same amount of cells independent of being alone or combined with other metrics can be disregarded as they have little effect on the overall outcome of the filtering.

In [ ]:
_ = pl.qc_filter.upset_plot_filter_impacts(adata, 
                                           thresholds=initial_thresholds, 
                                           groupby=groupby,
                                           limit_combinations=limit_combinations)

In [ ]:
%matplotlib widget

# Plot violins and sliders
obs_figure, obs_slider_dict = pl.qc_filter.quality_violin(
    adata,
    columns=obs_columns,
    groupby=condition_column,
    which="obs",
    thresholds=initial_thresholds,
    global_threshold=global_threshold,
    title="Cell quality control (before)",
    save="cell_filtering.png"
)
obs_figure

#### 4.7.2 - Apply cell filtering

In [ ]:
# Get final thresholds
final_thresholds = pl.qc_filter.get_slider_thresholds(obs_slider_dict)
tools.qc_filter.thresholds_as_table(final_thresholds) # show thresholds

In [ ]:
%matplotlib inline
plt.close()  # close previous figure

_ = pl.qc_filter.upset_plot_filter_impacts(adata, 
                                           thresholds=final_thresholds, 
                                           groupby=groupby,
                                           limit_combinations=limit_combinations)

In [ ]:
# Show pairwise comparisons of column values w/ thresholds (mean values in case thresholds are grouped)
if len(final_thresholds) > 1:
    mean_thresholds = tools.qc_filter.get_mean_thresholds(final_thresholds)
    _ = pl.general.pairwise_scatter(adata.obs, obs_columns, thresholds=mean_thresholds, save="cell_filtering_scatter.pdf")

In [ ]:
tools.qc_filter.apply_qc_thresholds(adata, final_thresholds)

# remove empty features after cell filtering
adata = adata[:, adata.X.sum(axis=0) > 0]

#### 4.7.3 - Show data after filtering

In [ ]:
%matplotlib inline 

#Plot violins and sliders
figure, slider_dict = pl.qc_filter.quality_violin(
    adata,
    columns=obs_columns,
    groupby=condition_column,
    which="obs", ncols=3,
    global_threshold = global_threshold,
    title="Cell quality control (after)",
    save="cell_filtering_final.pdf"
)
figure 

## 5 - Feature processing
<hr style="border:2px solid black"> </hr>

This section filters features (peaks). The user has the option to remove features located on the mitochondrial chromosome and features on either of the allosomes. Additionally, features can be reduced to highly variable features (features with high differences between cells).

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Removal of feature subsets
filter_chrM = True  # True or False; filtering out chrM
filter_xy = True    # True or False; filtering out chrX and chrY

# Highly Variable Features options 
select_highly_variable = False
min_cells = 5 # This one is mandatory
max_cells = None

__________

### 5.1 - Filter additional marked features

In [ ]:
if filter_chrM:
    print("Removing chromosomal features...")
    non_m = [name for name in adata.var_names if not name.startswith('chrM')]  # remove chrM
    adata = adata[:, non_m]
    
if filter_xy:
    print("Removing gender related features...")
    non_xy = [name for name in adata.var_names if not name.startswith('chrY') | name.startswith('chrX')]
    adata = adata[:, non_xy]

_________

### 5.2 - Select highly variable features

In [ ]:
# update number of cells per feature
adata = tools.qc_filter.calculate_qc_metrics(adata, var_type='features')

# drop total_counts as it is the same as n_features
adata.obs.drop(columns=["total_counts", "log1p_total_counts"], inplace=True)

if select_highly_variable:
    # get highly variable features
    tools.highly_variable.get_variable_features(adata, max_cells, min_cells)
    #Number of variable genes selected
    adata.var["highly_variable"].sum()
    # plot HVF violin
    pl.highly_variable.violin_HVF_distribution(adata)

________

## 6 - Save filtered adata
<hr style="border:2px solid black"> </hr>
Store the final results

In [ ]:
adata

In [ ]:
#Saving the data
adata_output = "anndata_2.h5ad"
utils.adata.save_h5ad(adata, adata_output)

In [ ]:
sctoolbox.settings.close_logfile()